In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = "openai/gpt-oss-20b")
llm_response  = llm.invoke("What is the capital of France?")
print(llm_response)

content='The capital of France is Paris.' additional_kwargs={'reasoning_content': 'We need to answer: "What is the capital of France?" The answer is Paris. Provide a concise answer.'} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 78, 'total_tokens': 118, 'completion_time': 0.041359315, 'completion_tokens_details': {'reasoning_tokens': 24}, 'prompt_time': 0.006290422, 'prompt_tokens_details': None, 'queue_time': 0.049717578, 'total_time': 0.047649737}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--6ea5e675-4607-491d-b645-8d0fb1cbf1d7-0' usage_metadata={'input_tokens': 78, 'output_tokens': 40, 'total_tokens': 118, 'output_token_details': {'reasoning': 24}}


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = llm | output_parser
parsed_response = chain.invoke("What is the capital of Germany?")
print(parsed_response)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


The capital of Germany is **Berlin**.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [11]:
from typing import List
from pydantic import BaseModel, Field

class RelocationDetails(BaseModel):
    city: str = Field(description="The city to which the person is relocating")
    country: str = Field(description="The country to which the person is relocating")
    from_city: str = Field(description="The city from which the person is relocating")
    from_country: str = Field(description="The country from which the person is relocating")
    recommended_neighborhoods: List[str] = Field(description="A list of recommended neighborhoods in the new city")
    average_cost_of_living: float = Field(description="The average monthly cost of living in the new city in USD")
    summary: str = Field(description="A brief summary of the relocation details")

relocation_text = """
A Candidate is living in london and is relocating to Berlin, Germany for a new job opportunity.
"""

structured_llm = llm.with_structured_output(RelocationDetails)
relocation_details = structured_llm.invoke(relocation_text)
print(relocation_details)

city='Berlin' country='Germany' from_city='London' from_country='United Kingdom' recommended_neighborhoods=['Mitte', 'Prenzlauer Berg', 'Neukölln', 'Charlottenburg'] average_cost_of_living=2500.0 summary='A candidate currently residing in London, United Kingdom, is relocating to Berlin, Germany for a new job opportunity. The average monthly cost of living in Berlin is estimated to be $2,500 USD. Recommended neighborhoods include Mitte, Prenzlauer Berg, Neukölln, and Charlottenburg.'


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
  "Provide a brief summary about {topic}."
)
chain = prompt | llm | output_parser
result = chain.invoke({"topic": "the Eiffel Tower"})
print(result)

The Eiffel Tower is a wrought‑iron lattice tower located on the Champ de Mars in Paris, France. Designed by engineer Gustave Eiffel and built for the 1889 Exposition Universelle, it stands 330 m (1,083 ft) tall and was the tallest man‑made structure in the world until 1930. Initially criticized by artists and architects, it has become an iconic symbol of Paris and a major tourist attraction, drawing millions of visitors each year. Its construction showcased French engineering prowess and remains a celebrated example of 19th‑century industrial architecture.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [14]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that provides concise information."),
    HumanMessage(content="Tell me a fun fact about the Great Wall of China.")
]

result = llm.invoke(messages)
print(result)

template = ChatPromptTemplate([
  ("system", "You are a knowledgeable assistant."),
  ("human", "Explain the significance of {event} in history.")
])

chain = template | llm
response = chain.invoke({"event": "the fall of the Berlin Wall"})
print(response)

content='Fun fact: The Great Wall stretches roughly 13,000 miles (about 21,000\u202fkm) – longer than the distance from the Earth to the Moon and back!' additional_kwargs={'reasoning_content': 'We need to provide a concise fun fact about the Great Wall. The user asks: "Tell me a fun fact about the Great Wall of China." They want a fun fact. The instruction from developer: "You are a helpful assistant that provides concise information." So we need to give a short, concise fun fact. Let\'s produce a brief sentence. For example: "The Great Wall stretches over 13,000 miles, longer than the distance from the Earth to the Moon and back." That is a fun fact. Or "The Great Wall was built by millions of workers over centuries, and its total length is about 21,196 km." But we need concise. We\'ll give one fun fact. Let\'s answer.'} response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 96, 'total_tokens': 286, 'completion_time': 0.198702987, 'completion_tokens_details': {'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


content='**The fall of the Berlin Wall—November\u202f9,\u202f1989—was one of the most symbolic and consequential events of the late 20th century. Its significance can be understood on several interconnected levels:**\n\n| Level | What happened | Why it mattered |\n|-------|---------------|-----------------|\n| **Symbolic** | The wall, built in 1961, physically separated East Berlin (Soviet‑aligned) from West Berlin (U.S., U.K., France‑aligned). | Its collapse became a vivid visual representation of the end of the “Iron Curtain” that had divided Europe and the world for nearly three decades. |\n| **Political** | The fall triggered the rapid collapse of the East German regime, the dissolution of the Warsaw Pact, and accelerated the collapse of the Soviet Union. | It demonstrated that the Soviet‑backed communist system could no longer maintain control over its satellite states, leading to a wave of democratic reforms across Eastern Europe. |\n| **Social** | Families and friends who had be

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
